# VROOM testing without ORS
updated with vroom 1.8.0 vroom-express 0.7, supporting Shipments

docs vroom iinput output formats:  https://github.com/VROOM-Project/vroom/blob/v1.8.0/docs/API.md

http://localhost/notebooks/tests/test5.ipynb 2 veicoli 1 solo mattina 2 corse

http://localhost/notebooks/tests/test6.ipynb 2 veicoli tutto il giorno 4 corse con priorita


In [35]:
import openrouteservice as ors
import requests
import folium
import json
import random
from pprint import pprint
from datetime import datetime
from datetime import date
import time
import polyline

vroom_url = 'http://vroom:3000/optimization/'
vehicleProfile = "driving-car"

# DATI posizioni
geoA = json.load(open('../data/mezzolombardo.geojson'))['features']   #4 points
geoB = json.load(open('../data/mezzocorona.geojson'))['features']     #6 points
geoC = json.load(open('../data/piana_rotaliana.geojson'))['features'] #6 points distanti

pointsA = [ p['geometry']['coordinates'] for p in geoA ]
pointsB = [ p['geometry']['coordinates'] for p in geoB ]
pointsC = [ p['geometry']['coordinates'] for p in geoC ]

#drt_example = json.load(open('../data/drt_2_veicoli_4_corse.json'))
drt_example = json.load(open('../data/drt_2_veicoli-fulltime_4_corse-priorita.json'))

for v in drt_example['vehicles']:
    v['profile'] = vehicleProfile

center = [
    11.1126,
    46.2083
]
zoom = 14

mapBbox = [11.077569,46.187199,11.161166,46.230027]

point_start =  [11.10249,46.21579] #posizione iniziale veicoli
point_start2 = [11.11258,46.20834]
point_end = point_start.copy()

def revcc(cc):
    return list(reversed(cc))

def toKm(m):
    return str(round(m/1000))+'km'

def toMin(sec):
    return str(round(sec/60))+'min'

def sec2hour(sec):
    return str(datetime.fromtimestamp(sec).strftime("%H:%M"))

def hour2sec(hour = '00:00'):
    now = datetime.now()
    s = now.strftime("%d/%m/%Y ") + hour
    return int(time.mktime(datetime.strptime(s, "%d/%m/%Y %H:%M").timetuple()))

def locShift(loc, s):
    return [loc[0] + s, loc[1] + s]

def randomPoint(bb):
    min_x, min_y, max_x, max_y = bb
    x = random.uniform(min_x, max_x)
    y = random.uniform(min_y, max_y)
    return [x, y]

def vehicle2Map(v):
    idv = str(v['id'])    
    loc = v['start']
    cap = str(v['capacity'][0])

    try:
        times = v['time_window']
    except:
        times = []
    
    mText = "VEICOLO %s" % idv
    #mText += "<br> capacity %s" % cap
    
    if len(times) > 0:
        mText += "<br>Orario turno <br> da %s a %s " % (sec2hour(times[0]), sec2hour(times[1]) )
    
    mColor = 'red'#COLORS[idv]
    
    mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right',permanent=True), icon=folium.Icon(color=mColor, icon='truck', prefix='fa'))
    mark.add_to(Map)

def job2Map(j):
    idj = str(j['id'])    
    loc = j['location']
    if hasattr(j,'time_windows'):
        times = j['time_windows'][0]
    else:
        times = []
    
    mText = "job %s<br>" % (idj)
    #+"quantity %s <br> delivery time %s" % (delivery, toMin(service))
    
    if len(times) > 0:
        mText += "da: %s <br> a: %s " % ( sec2hour(times[0]), sec2hour(times[1]) )

    mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(mText, direction='right', permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green'))
    mark.add_to(Map)

def ship2Map(s):

    color = COLORS[str(s['pickup']['id'])]
    
    # prenotazione PARTENZA
    loc1 = s['pickup']['location']
    mText = "ID %s" % s['pickup']['id']
    if s['priority']>0:
        mText +=" (CONFERMATA)"
    
    try:
        times = s['pickup']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "<br>Partenza orario <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )
    
    #mText += "Quantita %s <br>" % s['amount'][0]
    #mText += "Priority %s <br>" % s['priority']
    
    mTip = folium.Tooltip(mText, direction='right', permanent=True) 
    mIcon = folium.Icon(color=color,prefix='fa')
    mark = folium.Marker(revcc(loc1), tooltip=mTip, icon=mIcon)
    mark.add_to(Map)
    
    # prenotazione ARRIVO
    loc2 = s['delivery']['location']
    mText = "ID %s <br>" % s['delivery']['id']
    try:
        times = s['delivery']['time_windows'][0]
    except:
        times = []
    
    if len(times) > 0:
        mText += "Arrivo orario <br> %s a %s <br>" % ( sec2hour(times[0]), sec2hour(times[1]) )

    #mText += "Quantita %s <br>" % s['amount'][0]
    
    folium.PolyLine(locations = [revcc(loc1),revcc(loc2)], color = color, opacity = 0.6, weight = 3).add_to(Map)
    
    mTip = folium.Tooltip(mText, direction='right', permanent=True)
    mIcon = folium.Icon(color=color,prefix='fa')
    mark = folium.Marker(revcc(loc2), tooltip=mTip, icon=mIcon)
    mark.add_to(Map)

def step2Map(step):
    sid = ''
    wait = 0
    try:
        loc = step['location']
        sid = step['id']
        wait = step['waiting_time']
    except:
        loc = None

    if loc and step['type'] != 'end':
        tit = str(i)+" "+step['type']+"<br> arrivo: %s " % sec2hour(step['arrival'])

        mark = folium.Marker(revcc(loc), tooltip=folium.Tooltip(tit, direction='bottom', permanent=True, style='background:yellow'), icon=folium.Icon(icon='circle', prefix='fa', color='green'))
        mark.add_to(Map)

def step2Print(step):
    sid = ''
    wait = 0
    try:
        loc = step['location']
        sid = step['id']
        wait = step['waiting_time']
    except:
        loc = None

    if sid:
        sText = "CORSA %s alle %s - %s" % ( sid, sec2hour(step['arrival']), step['type'] )
    else:
        sText = "%s alle %s " % (step['type'].upper(), sec2hour(step['arrival']) )

    if wait :
        sText += " - Attesa %s " % str(sec2hour(wait))
    
    print(sText)

#CUSTOM ICONS https://fontawesome.com/icons?d=gallery&p=2

Map = folium.Map(location = revcc(center), tiles='OpenStreetMap', zoom_start = zoom)

COLORS = {
    '1':'blue','2':'darkred','3':'purple','4':'green',
    '5':'orange','6':'cadetblue','7':'beige','8':'pink',
    '9':'darkblue','10':'darkgreen'
}

HOURS1 = [
[hour2sec('08:00'),hour2sec('08:05')],
[hour2sec('09:00'),hour2sec('09:05')],
[hour2sec('10:00'),hour2sec('10:05')],
[hour2sec('11:00'),hour2sec('11:05')],
[hour2sec('12:00'),hour2sec('12:05')]
]

HOURS2 = [
[hour2sec('14:00'),hour2sec('14:05')],
[hour2sec('15:00'),hour2sec('15:05')],
[hour2sec('16:00'),hour2sec('16:05')],
[hour2sec('17:00'),hour2sec('17:05')],
[hour2sec('18:00'),hour2sec('18:05')]
]
# VEICOLI

HOURSall = HOURS1 + HOURS2

vehicles = []

### VEICOLO 1
vid = 1

loc = locShift(point_start, 0.004)

vehicle = {
    'id': vid,
    'capacity': [1],    # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,       # posizione di partenza veicoli
    'end': loc,
    'time_window': [              #orario lavoro del veicolo 1
        hour2sec('07:00'),
        hour2sec('19:00')
    ],
#     'breaks': [
#         {
#           "id": 1,
#           "service": 300,   #durata pausa 5 minuti
#           "time_windows": [ #an array of time_window objects describing valid slots for break start
#             [hour2sec('13:05'),hour2sec('13:55')],
#           ]
#         }
#     ]
}
vehicles.append(vehicle)

### VEICOLO 2
vid = 2

loc = drt_example['vehicles'][1]['start']

vehicle = {
    'id': vid,
    'capacity': [1],     # Limite di capacita' del Veicolo    
    'profile': vehicleProfile,
    'start': loc,        # posizione di partenza veicoli
    'end': loc,
    'time_window': [                  # pomeridiano
        hour2sec('07:00'),
        hour2sec('19:00')
    ]
}
vehicles.append(vehicle)

points = pointsA + pointsB

shipsProgrammed = [1]    # SIMULA ELENCO PRENOTAZIONI PROGRAMMATE( elenco id prenotazioni)

shipments = []

genships = 4

for i in list(range(genships)):
    
    sid = i+1
    
    ship = {          #struttura minima valida per prenotazione
        'pickup': {
            'id': sid,
            'location': randomPoint( mapBbox )
        },
        'delivery': {
            'id': sid,
            'location': randomPoint( mapBbox )
        },        
        'priority': 0,
        'amount': [ 1 ]        
    }
    
    if sid in shipsProgrammed:
        ship['priority']= 100;
        
    shiptype = random.choice(['pickup','delivery']) # pickup == PARTENZA, delivery == ARRIVO
    
    ship[ shiptype ]['time_windows'] = [ random.choice(HOURSall) ]
    
    shipments.append(ship)

request = {
     'vehicles': [vehicles[0]],
     'shipments': shipments,
     'jobs': [],
}

# RENDERIZZA PROBLEMA RICHIESTO IN MAP
for v in request['vehicles']:
    vehicle2Map(v)

for s in request['shipments']:
    
    ship2Map(s)

    try:
        timeStart = sec2hour(s['pickup']['time_windows'][0][0])
    except:
        timeStart = '     '
    
    try:
        timeEnd = sec2hour(s['delivery']['time_windows'][0][0])
    except:
        timeEnd = '     '
    
    shipText = "CORSA %s - partenza %s arrivo %s" % (s['pickup']['id'], timeStart, timeEnd)
    print(shipText)

Map

CORSA 1 - partenza       arrivo 18:00
CORSA 2 - partenza 18:00 arrivo      
CORSA 3 - partenza       arrivo 14:00
CORSA 4 - partenza 12:00 arrivo      


In [36]:
request['geometry'] = True
resp = requests.post(vroom_url, json = request)
solution = resp.json()

for v in request['vehicles']:
    vehicle2Map(v)

print(len(shipments), "CORSE richieste\n")
print("CORSE programmate:", shipsProgrammed )

if len(solution['routes']) > 0:
    for route in solution['routes']:
        
        rtext = "distance: %s delivered quantity: %s pickup quantity: %s "  % ( toKm(route['distance']), route['delivery'][0], route['pickup'][0] )
        
        rtext = rtext+", durata: %s " % toMin(route['duration']+route['service'])
        
        rcolor = COLORS[str(route['vehicle'])]
        #rcolor = random.choice( list(COLORS.values()) )
        
        print("\nVehicle "+str(route['vehicle']), rcolor)
        print(rtext)
        
        geomdecoded = ors.convert.decode_polyline(route['geometry'])
        route['geometry'] = geomdecoded
        folium.PolyLine(
            locations = [revcc(coords) for coords in geomdecoded['coordinates']],
            color = rcolor,
            opacity = 0.8,
            weight = 5
        ).add_to(Map)
        
        for step in route['steps']:
            step2Print(step)
            step2Map(step)

    print("\nCORSE NON ASSEGNATE:")
    for un in solution['unassigned']:
        print('CORSA '+str(un['id']))
    
    print("\nTotal distance: "+toKm(solution['summary']['distance']), 'Not Delivered: '+str(len(solution['unassigned'])))
    
Map

4 CORSE richieste

CORSE programmate: [1]

Vehicle 1 blue
distance: 39km delivered quantity: 4 pickup quantity: 4 , durata: 47min 
START alle 11:58 
CORSA 4 alle 12:05 - pickup
CORSA 4 alle 12:09 - delivery
CORSA 3 alle 12:14 - pickup
CORSA 3 alle 12:22 - delivery - Attesa 01:37 
CORSA 1 alle 14:06 - pickup
CORSA 1 alle 14:12 - delivery - Attesa 03:47 
CORSA 2 alle 18:00 - pickup
CORSA 2 alle 18:06 - delivery
END alle 18:11 

CORSE NON ASSEGNATE:

Total distance: 39km Not Delivered: 0


In [22]:
#print(json.dumps(solution, indent=2))

for route in solution['routes']:
    print(json.dumps(route['steps'], indent=2))

[
  {
    "type": "start",
    "location": [
      11.10649,
      46.21979
    ],
    "load": [
      0
    ],
    "arrival": 1619780246,
    "duration": 0,
    "distance": 0
  },
  {
    "type": "pickup",
    "location": [
      11.095685196154584,
      46.21990053069435
    ],
    "id": 3,
    "service": 0,
    "waiting_time": 0,
    "job": 3,
    "load": [
      1
    ],
    "arrival": 1619780351,
    "duration": 105,
    "distance": 884
  },
  {
    "type": "delivery",
    "location": [
      11.13620378852386,
      46.21336468202394
    ],
    "id": 3,
    "service": 0,
    "waiting_time": 0,
    "job": 3,
    "load": [
      0
    ],
    "arrival": 1619780700,
    "duration": 454,
    "distance": 4765
  },
  {
    "type": "pickup",
    "location": [
      11.143385851747004,
      46.18973319214656
    ],
    "id": 1,
    "service": 0,
    "waiting_time": 2920,
    "job": 1,
    "load": [
      1
    ],
    "arrival": 1619781080,
    "duration": 834,
    "distance": 8620
  },
